In [2]:
import json
import numpy as np
import faiss
import openai
import os
import tiktoken
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=api_key)

# Load FAISS index and metadata
index = faiss.read_index("faiss.index")
with open("faiss_metadata.json", "r") as f:
    metadata = json.load(f)

# Tokenizers
encoding = tiktoken.encoding_for_model("gpt-4")
ADA_ENCODING = tiktoken.encoding_for_model("text-embedding-ada-002")

def count_tokens(text, model_encoding=encoding):
    return len(model_encoding.encode(text))

# Embed query
def embed_query(query):
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=query
    )
    return np.array([response.data[0].embedding], dtype="float32")

# Apply filters to metadata
def apply_filters(results, year=None, program=None, rule_type=None):
    filtered = []
    for item in results:
        meta = item["metadata"]
        if year and str(meta.get("year")) != str(year):
            continue
        if program and program.lower() not in meta.get("title", "").lower():
            continue
        if rule_type and rule_type.lower() not in meta.get("rule_type", "").lower():
            continue
        filtered.append(item)
    return filtered

# Build the GPT-4 prompt
def build_prompt(query, results, prompt_token_limit=5000):
    context = ""
    for result in results:
        chunk = result["text"]
        header = result["section_header"]
        next_section = f"\n[Section: {header}]\n{chunk}\n"
        if count_tokens(context + next_section) > prompt_token_limit:
            break
        context += next_section

    prompt = f"""You are a regulatory analysis expert specializing in U.S. healthcare policy rules.

Using the information provided below, write a comprehensive and detailed answer to the user's question.

Instructions:
- Use bullet points or numbered lists where helpful.
- Reference section headers in square brackets (e.g., [Section: III.A.3]).
- Highlight policy changes, comparisons, financial impacts, and dates when applicable.
- Base your answer solely on the provided context.

--- BEGIN CONTEXT ---
{context}
--- END CONTEXT ---

User Question:
{query}

Answer:
"""
    return prompt

# Ask GPT-4 and format result
def ask_gpt4(prompt, max_completion_tokens=2500):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful AI that answers questions about Medicare rules."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=max_completion_tokens
    )
    content = response.choices[0].message.content
    # Inline highlighting of section headers
    return content.replace("[Section:", "**[Section:").replace("]", "]**")

# Main query function
def search_and_ask(query, k=12, year=None, program=None, rule_type=None):
    print(f"\n🔍 Embedding query: {query}")
    query_embedding = embed_query(query)
    distances, indices = index.search(query_embedding, k)
    initial_results = [metadata[i] for i in indices[0]]

    # Apply filters
    results = apply_filters(initial_results, year=year, program=program, rule_type=rule_type)
    if not results:
        print("⚠️ No results found after applying filters.")
        return

    prompt = build_prompt(query, results)
    print("\n🧠 Asking GPT-4...")
    answer = ask_gpt4(prompt)
    print("\n✅ Answer:\n")
    print(answer)

# Command-line runner
if __name__ == "__main__":
    try:
        user_query = input("Enter your question: ")
        year = input("Filter by year (or leave blank): ").strip() or None
        program = input("Filter by program (Hospice, SNF, MPFS): ").strip() or None
        rule_type = input("Filter by rule type (Final, Proposed): ").strip() or None
        search_and_ask(user_query, k=12, year=year, program=program, rule_type=rule_type)
    except Exception as e:
        print(f"❌ Error: {e}")




🔍 Embedding query: What changed in final hospice payment between 2024 and 2025?

🧠 Asking GPT-4...

✅ Answer:

Several changes were made in the final hospice payment between 2024 and 2025:

1. **Hospice Payment Update Percentage**: The hospice payment update percentage for FY 2025 was increased to 2.9 percent, up from the proposed 2.6 percent **[Section: III. Provisions of the Final Rule > A. Final FY 2025 Hospice Wage Index and Rate Update > 3. FY 2025 Hospice Payment Update Percentage]**. This update is based on a 3.4 percent inpatient hospital market basket percentage increase, reduced by a final 0.5 percentage point productivity adjustment **[Section: V. Regulatory Impact Analysis > C. Detailed Economic Analysis > 1. Hospice Payment Update for FY 2025]**.

2. **Hospice Cap Amount**: The hospice cap amount for FY 2025 was updated by the hospice payment update percentage of 2.9 percent **[Section: III. Provisions of the Final Rule > A. Final FY 2025 Hospice Wage Index and Rate Updat